In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from tensorflow import keras


from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


from keras.layers import SimpleRNN, Embedding, Dense, LSTM, Dropout, SimpleRNN, CuDNNLSTM
from keras.models import Sequential






Using TensorFlow backend.


In [0]:
!pip list

In [2]:
from google.colab import files
uploaded = files.upload()


Saving dataset (1).csv to dataset (1).csv


In [4]:
import io
inputs = pd.read_csv(io.StringIO(uploaded['dataset (1).csv'].decode('utf-8')))
inputs.head()
print(len(inputs))

1600


In [5]:
inputs = inputs.dropna()
print(len(inputs))
print(inputs)

1600
      index   Decision  Sentiment  ...   n count  wrd count  lstm
0      1234  deceptive          0  ...  0.188119   0.393939     0
1       136   truthful          1  ...  0.262712   0.348485     1
2       202   truthful          1  ...  0.205128   0.212121     1
3      1261  deceptive          0  ...  0.232394   0.242424     0
4      1278  deceptive          0  ...  0.255319   0.212121     0
...     ...        ...        ...  ...       ...        ...   ...
1595   1279  deceptive          0  ...  0.200000   0.287879     0
1596   1255  deceptive          0  ...  0.214286   0.242424     0
1597    719  deceptive          0  ...  0.258741   0.318182     0
1598    210   truthful          1  ...  0.153846   0.121212     0
1599    918   truthful          1  ...  0.236392   0.636364     0

[1600 rows x 16 columns]


In [6]:
df2 = inputs.reindex(np.random.permutation(inputs.index))
print(df2)

      index   Decision  Sentiment  ...   n count  wrd count  lstm
1281   1412  deceptive          0  ...  0.233766   0.151515     0
129    1256  deceptive          0  ...  0.181818   0.151515     0
241     989   truthful          1  ...  0.193396   0.378788     1
337    1113   truthful          1  ...  0.221053   0.075758     1
1063    596  deceptive          0  ...  0.316940   0.363636     0
...     ...        ...        ...  ...       ...        ...   ...
1485    197   truthful          1  ...  0.221477   0.303030     1
1207    390   truthful          1  ...  0.200000   0.242424     1
1127     66   truthful          1  ...  0.275943   0.681818     1
140      70   truthful          1  ...  0.235955   0.181818     1
1483   1482  deceptive          0  ...  0.278146   0.151515     0

[1600 rows x 16 columns]


In [7]:
labels = df2['Decision'].values.tolist()
print(labels)

['deceptive', 'deceptive', 'truthful', 'truthful', 'deceptive', 'truthful', 'truthful', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'truthful', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'deceptive', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'truthful', 'truthful', 'deceptive', 'deceptive', 'deceptive', 'truthful', 'truthful', 'truthful', 'truthful', 'deceptive', 'deceptive', 'truthful', 'truthful', 'truthful', 'deceptive', 'truthful', 'truthful', 'truthful', 'deceptive', 'truthful', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'truthful', 'truthful', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'tru

In [0]:
labels = np.array([0 if label == "deceptive" else 1 for label in labels])

In [0]:

labels = df2['Sentiment'].to_numpy()

In [9]:

print(len(labels))

print(labels)

print(labels.shape)


1600
[0 0 1 ... 1 1 0]
(1600,)


In [10]:
print(labels)

[0 0 1 ... 1 1 0]


In [11]:
del df2['Sentiment']
del df2['index']
del df2['Decision']
print(df2)


         anger      fear  anticipation  ...   n count  wrd count  lstm
1281  0.025974  0.012987      0.051948  ...  0.233766   0.151515     0
129   0.037736  0.018868      0.018868  ...  0.181818   0.151515     0
241   0.018868  0.014151      0.028302  ...  0.193396   0.378788     1
337   0.010526  0.010526      0.052632  ...  0.221053   0.075758     1
1063  0.000000  0.005495      0.032967  ...  0.316940   0.363636     0
...        ...       ...           ...  ...       ...        ...   ...
1485  0.000000  0.006667      0.026667  ...  0.221477   0.303030     1
1207  0.000000  0.000000      0.025000  ...  0.200000   0.242424     1
1127  0.002392  0.004785      0.019139  ...  0.275943   0.681818     1
140   0.000000  0.000000      0.000000  ...  0.235955   0.181818     1
1483  0.026490  0.019868      0.026490  ...  0.278146   0.151515     0

[1600 rows x 13 columns]


In [12]:
train_x = df2.to_numpy()
print(len(train_x))
a = train_x[0]
print(a)

1600
[0.02597403 0.01298701 0.05194805 0.03896104 0.02597403 0.01298701
 0.03896104 0.02597403 0.12987013 0.02597403 0.23376623 0.151515
 0.        ]


In [0]:
split_frac1 = 0.8

idx1 = int(len(train_x) * split_frac1)
train_x, val_x = train_x[:idx1], train_x[idx1:]
train_y, val_y = labels[:idx1], labels[idx1:]

In [14]:
#print(train_x)
print(len(train_x))
print(len(train_y))
print(len(val_x))
print(len(val_y))

1280
1280
320
320


In [15]:
print(val_x)

[[0.         0.00952381 0.00952381 ... 0.2293578  0.106061   1.        ]
 [0.0625     0.03125    0.         ... 0.1875     0.090909   0.        ]
 [0.03365385 0.01442308 0.00961539 ... 0.20853081 0.439394   0.        ]
 ...
 [0.00239234 0.00478469 0.01913876 ... 0.2759434  0.681818   1.        ]
 [0.         0.         0.         ... 0.23595506 0.181818   1.        ]
 [0.02649007 0.01986755 0.02649007 ... 0.2781457  0.151515   0.        ]]


In [0]:
clf = LogisticRegression()
clf.fit(train_x,train_y)
p = clf.predict(val_x)
print (accuracy_score(train_y,p))

Basic NN
 

In [18]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64,activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(rate=0.25))

model.add(tf.keras.layers.Dense(64,activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(rate=0.25))

model.add(tf.keras.layers.Dense(64,activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(rate=0.25))

model.add(tf.keras.layers.Dense(64,activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(rate=0.25))

model.add(tf.keras.layers.Dense(1,activation=tf.nn.sigmoid))


model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

class TestCallback(tf.keras.callbacks.Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))



#model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs=50, callbacks=[TestCallback((val_x,val_y))])



model.fit(train_x,train_y,epochs=50,batch_size=64,verbose=1)
# _, accuracy = model.evaluate(val_x, val_y)
# print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/50
20/20 [==============================] - 0s 2ms/step - loss: 0.5315 - accuracy: 0.8250
Epoch 2/50
20/20 [==============================] - 0s 2ms/step - loss: 0.2771 - accuracy: 0.9484
Epoch 3/50
20/20 [==============================] - 0s 2ms/step - loss: 0.2166 - accuracy: 0.9484
Epoch 4/50
20/20 [==============================] - 0s 2ms/step - loss: 0.2154 - accuracy: 0.9484
Epoch 5/50
20/20 [==============================] - 0s 2ms/step - loss: 0.2122 - accuracy: 0.9492
Epoch 6/50
20/20 [==============================] - 0s 2ms/step - loss: 0.2036 - accuracy: 0.9492
Epoch 7/50
20/20 [==============================] - 0s 2ms/step - loss: 0.2193 - accuracy: 0.9492
Epoch 8/50
20/20 [==============================] - 0s 2ms/step - loss: 0.2140 - accuracy: 0.9492
Epoch 9/50
20/20 [==============================] - 0s 2ms/step - loss: 0.2164 - accuracy: 0.9492
Epoch 10/50
20/20 [==============================] - 0s 2ms/step - loss: 0.2162 - accuracy: 0.9492
Epoch 11/50
20/20 [

In [19]:

_, accuracy = model.evaluate(val_x, val_y)
print('Accuracy: %.2f' % (accuracy*100))

10/10 [==============================] - 0s 2ms/step - loss: 0.1303 - accuracy: 0.9750
Accuracy: 97.50


In [20]:
y_preds = model.predict(val_x)
print(y_preds)

[[0.97025114]
 [0.1082442 ]
 [0.03790569]
 [0.9721548 ]
 [0.05961041]
 [0.04522159]
 [0.9676097 ]
 [0.9656273 ]
 [0.06412045]
 [0.09272891]
 [0.05017214]
 [0.96914893]
 [0.11703889]
 [0.06847395]
 [0.03707579]
 [0.09324469]
 [0.09579463]
 [0.06151437]
 [0.05241559]
 [0.956441  ]
 [0.03777761]
 [0.04360061]
 [0.08948414]
 [0.05467585]
 [0.9631181 ]
 [0.0349156 ]
 [0.06435816]
 [0.04721694]
 [0.05379654]
 [0.9545785 ]
 [0.9655906 ]
 [0.96124583]
 [0.05121782]
 [0.04459904]
 [0.07018679]
 [0.03557991]
 [0.06032001]
 [0.9630784 ]
 [0.96747667]
 [0.9681701 ]
 [0.07140552]
 [0.95612144]
 [0.03156948]
 [0.0586984 ]
 [0.9625585 ]
 [0.04090835]
 [0.96461064]
 [0.9689154 ]
 [0.96142215]
 [0.9636531 ]
 [0.06840194]
 [0.10904489]
 [0.03607921]
 [0.06435747]
 [0.06985013]
 [0.07027598]
 [0.96361965]
 [0.96678835]
 [0.96772915]
 [0.05180653]
 [0.06622338]
 [0.10229431]
 [0.05927318]
 [0.04356524]
 [0.9659086 ]
 [0.06894415]
 [0.0494414 ]
 [0.94982755]
 [0.0426826 ]
 [0.05472907]
 [0.03211556]
 [0.96

In [21]:

for i in range(320):
  if(y_preds[i]>=0.5):
    y_preds[i]=1
  else:
    y_preds[i]=0

print(y_preds)

[[1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.

In [22]:
from sklearn import metrics
print('Test accuracy: {:.2f} %'.format(100*metrics.accuracy_score(y_preds, val_y)))

Test accuracy: 97.50 %


In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_5 (Dense)              multiple                  896       
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
dense_6 (Dense)              multiple                  4160      
_________________________________________________________________
dropout_5 (Dropout)          multiple                  0         
_________________________________________________________________
dense_7 (Dense)              multiple                  4160      
_________________________________________________________________
dropout_6 (Dropout)          multiple                 